In [ ]:
import time
t1 = time.time()

# Setup

## Google Drive


In [ ]:
# from google.colab import drive
# drive.mount('/content/gdrive')

In [ ]:
# cd "/content/gdrive/My Drive/Optiver Kaggle/Data"

In [ ]:
# output_file = r'/content/gdrive/My Drive/Optiver Kaggle/\working/submission.csv'
# train_file = r'/content/gdrive/My Drive/Optiver Kaggle/working/train.csv'

## Kaggle

In [ ]:
cd "/kaggle/input/optiver-realized-volatility-prediction/"

In [ ]:
output_file = r'/kaggle/working/submission.csv'
train_file = r'/kaggle/working/train.csv'

## PC

In [ ]:
# cd D:\kaggle\input\optiver-realized-volatility-prediction

In [ ]:
# output_file = r'D:\kaggle\working\submission.csv'
# train_file = r'D:\kaggle\working\train.csv'

# Libraries

In [ ]:
import pandas as pd
import numpy as np
import glob
import os
import gc

from joblib import Parallel, delayed

from sklearn import preprocessing, model_selection
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import QuantileTransformer
from sklearn.metrics import r2_score

import matplotlib.pyplot as plt 
import seaborn as sns
import numpy.matlib

seed0 = 2021
target_name = 'target'
scores_folds = {}

pd.set_option('display.max_rows',None)
pd.set_option('display.max_columns',None)

# Data Preprocess

## Read Target Data 

In [ ]:
def read_train_test():
    train = pd.read_csv('train.csv')
    test = pd.read_csv('test.csv')
    # Create a key to merge with book and trade data
    train['row_id'] = train['stock_id'].astype(str) + '-' + train['time_id'].astype(str)
    test['row_id'] = test['stock_id'].astype(str) + '-' + test['time_id'].astype(str)
    print(f'Our training set has {train.shape[0]} rows')
    return train, test

In [ ]:
# train,test = read_train_test()

In [ ]:
# train.head(5)

## Book and Trade Data Preprocess



### Pre functions

In [ ]:
# calculate weighted average price
def wap1(df):
    wap = (df['bid_price1'] * df['ask_size1'] + df['ask_price1'] * df['bid_size1']) / (df['bid_size1'] + df['ask_size1'])
    return wap

def wap2(df):
    wap = (df['bid_price2'] * df['ask_size2'] + df['ask_price2'] * df['bid_size2']) / (df['bid_size2'] + df['ask_size2'])
    return wap

def wap3(df):
    wap = (df['bid_price1'] * df['bid_size1'] + df['ask_price1'] * df['ask_size1']) / (df['bid_size1'] + df['ask_size1'])
    return wap

def wap4(df):
    wap = (df['bid_price2'] * df['bid_size2'] + df['ask_price2'] * df['ask_size2']) / (df['bid_size2'] + df['ask_size2'])
    return wap

# calculate log return
def log_return(series):
    return np.log(series).diff()

# calculate the realized volatility
def realized_volatility(series):
    return np.sqrt(np.sum(series**2))

# calculate the realized volatility
def realized_volatility_constant(df, col_name, new_col):
    df[new_col] = np.sqrt(np.sum(df[col_name]**2))
    return df

# count unique elements of a series
def count_unique(series):
    return len(np.unique(series))

# count unique elements of a series
def count_unique_constant(df, col_name, new_col):
    df[new_col] = len(np.unique(df[col_name]))
    return df

# calculate order flow using tick method
def order_flow_tick(df):
    df['order_flow_tick'] = np.nan
    df['price_difference'] = df['price'].diff()
    df.loc[df['price_difference']>0,'order_flow_tick'] = df['size']
    df.loc[df['price_difference']<0,'order_flow_tick'] = -df['size']
    # for those with price not change, assign a same sign with the previous value
    constant_index = df.index[df['price_difference']==0].values
    for index in constant_index:
        df.loc[index,'order_flow_tick'] = np.sign(df.loc[index-1,'order_flow_tick']) * df.loc[index,'size']
    df.drop('price_difference',inplace=True,axis=1)
    return df

def order_flow_quote(df,df_ob):
    # select the time id of orderbook to be consistent with trade
    time_id = df['time_id'].iloc[0]
    df_ob = df_ob.loc[df_ob['time_id']==time_id,:]

    df_ob['mid_price'] = (df_ob['ask_price1'] + df_ob['bid_price1']) / 2
    df = df.merge(df_ob[['seconds_in_bucket','mid_price']],on='seconds_in_bucket',how='left')
    # if there are none values, let the quote equals to tick
    df['order_flow_quote'] = df['order_flow_tick']
    df.loc[df['price']<df['mid_price'],'order_flow_quote'] = -df['size']
    df.loc[df['price']>df['mid_price'],'order_flow_quote'] = df['size']
    df.loc[df['price']==df['mid_price'],'order_flow_quote'] = 0

    df.drop('mid_price',inplace=True,axis=1)

    return df

def tendency_factors(df):
    def tendency(price, vol):    
        df_diff = np.diff(price)
        val = (df_diff/price[1:])*100
        power = np.sum(val*vol[1:])
        return power

    lis = []
    for n_time_id in df['time_id'].unique():
        df_id = df[df['time_id'] == n_time_id]        
        tendencyV = tendency(df_id['price'].values, df_id['size'].values)      
        f_max = np.sum(df_id['price'].values > np.mean(df_id['price'].values))
        f_min = np.sum(df_id['price'].values < np.mean(df_id['price'].values))
        df_max =  np.sum(np.diff(df_id['price'].values) > 0)
        df_min =  np.sum(np.diff(df_id['price'].values) < 0)
        # new
        abs_diff = np.median(np.abs( df_id['price'].values - np.mean(df_id['price'].values)))        
        energy = np.mean(df_id['price'].values**2)
        iqr_p = np.percentile(df_id['price'].values,75) - np.percentile(df_id['price'].values,25)

        # vol vars

        abs_diff_v = np.median(np.abs( df_id['size'].values - np.mean(df_id['size'].values)))        
        energy_v = np.sum(df_id['size'].values**2)
        iqr_p_v = np.percentile(df_id['size'].values,75) - np.percentile(df_id['size'].values,25)

        lis.append({'time_id':n_time_id,'tendency':tendencyV,'f_max':f_max,'f_min':f_min,'df_max':df_max,'df_min':df_min,
                    'abs_diff':abs_diff,'energy':energy,'iqr_p':iqr_p,'abs_diff_v':abs_diff_v,'energy_v':energy_v,'iqr_p_v':iqr_p_v})

    df_lr = pd.DataFrame(lis)

    return df_lr

### Main functions

#### Book

In [ ]:
# preprocess book data (for each stock id)
def book_preprocessor(file_path):
    df = pd.read_parquet(file_path)

    # calculate weighted average price
    df['wap1'] = wap1(df)
    df['wap2'] = wap2(df)
    df['wap3'] = wap3(df)
    df['wap4'] = wap4(df)
    # calculate log returns
    df['log_return1'] = df.groupby(['time_id'])['wap1'].apply(log_return)
    df['log_return2'] = df.groupby(['time_id'])['wap2'].apply(log_return)
    df['log_return3'] = df.groupby(['time_id'])['wap3'].apply(log_return)
    df['log_return4'] = df.groupby(['time_id'])['wap4'].apply(log_return)
    # calculate wap balance
    df['wap_gap1'] = abs(df['wap1'] - df['wap2'])
    df['wap_gap2'] = abs(df['wap3'] - df['wap4'])

    # calculate spread
    df['price_spread1'] = (df['ask_price1'] - df['bid_price1']) / ((df['ask_price1'] + df['bid_price1']))
    df['price_spread2'] = (df['ask_price2'] - df['bid_price2']) / ((df['ask_price2'] + df['bid_price2']))
    df['bid_spread'] = df['bid_price1'] - df['bid_price2']
    df['ask_spread'] = df['ask_price2'] - df['ask_price1']
    df['bid_ask_spread1'] = df['ask_price1'] - df['bid_price1']
    df['bid_ask_spread2'] = df['ask_price2'] - df['bid_price2']
    df["bid_ask_spread_spread"] = abs(df['bid_spread'] - df['ask_spread'])

    # calculate imbalance
    df['volume1'] = (df['ask_size1']) + (df['bid_size1'])
    df['volume_imbalance1'] = (df['ask_size1']) - (df['bid_size1'])
    df['imbalance_percentage1'] = df['volume_imbalance1'] / df['volume1']
    df['volume2'] = (df['ask_size2']) + (df['bid_size2'])
    df['volume_imbalance2'] = (df['ask_size2']) - (df['bid_size2'])
    df['imbalance_percentage2'] = df['volume_imbalance2'] / df['volume2']
    df['total_volume'] = (df['ask_size1'] + df['ask_size2']) + (df['bid_size1'] + df['bid_size2'])
    df['total_volume_imbalance'] = (df['ask_size1'] + df['ask_size2']) - (df['bid_size1'] + df['bid_size2'])
    df['total_imbalance_percentage'] = df['total_volume_imbalance'] / df['total_volume']

    # create dict of functions for each feature for aggregations
    create_feature_dict = {
      'wap1': [np.sum, np.std],
      'wap2': [np.sum, np.std],
      'wap3': [np.sum, np.std],
      'wap4': [np.sum, np.std],
      'log_return1': [realized_volatility],
      'log_return2': [realized_volatility],
      'log_return3': [realized_volatility],
      'log_return4': [realized_volatility],
      'wap_gap1': [np.sum, np.max],
      'wap_gap2': [np.sum, np.max],
      'price_spread1':[np.sum, np.max],
      'price_spread2':[np.sum, np.max],
      'bid_spread':[np.sum, np.max],
      'ask_spread':[np.sum, np.max],
      'bid_ask_spread1':[np.sum, np.max],
      'bid_ask_spread2':[np.sum, np.max],
      'bid_ask_spread_spread':[np.sum, np.max],
      'volume1':[np.sum, np.max],
      'volume_imbalance1':[np.sum, np.max],
      "imbalance_percentage1":[np.sum,  np.max],
      "volume2":[np.sum,  np.max],
      "volume_imbalance2":[np.sum,  np.max],
      "imbalance_percentage2":[np.sum,  np.max],
      "total_volume":[np.sum,  np.max],
      "total_volume_imbalance":[np.sum,  np.max],
      "total_imbalance_percentage":[np.sum,  np.max],
    }
    create_feature_dict_time = {
      'log_return1': [realized_volatility],
      'log_return2': [realized_volatility],
      'log_return3': [realized_volatility],
      'log_return4': [realized_volatility],
    }

    # Function to get group stats for different windows (seconds in bucket)
    def get_stats_window(fe_dict,seconds_in_bucket, add_suffix = False):
        # Group by the window
        df_feature = df[df['seconds_in_bucket'] >= seconds_in_bucket].groupby(['time_id']).agg(fe_dict).reset_index()
        # Rename columns joining suffix (get wap_sum from the multi index wap and sum)
        df_feature.columns = ['_'.join(col) for col in df_feature.columns]
        # Add a suffix to differentiate windows
        if add_suffix:
            df_feature = df_feature.add_suffix('_' + str(seconds_in_bucket))
        return df_feature

    # Get the stats for different windows
    df_feature = get_stats_window(create_feature_dict,seconds_in_bucket = 0, add_suffix = False)
    df_feature_500 = get_stats_window(create_feature_dict_time,seconds_in_bucket = 500, add_suffix = True)
    df_feature_400 = get_stats_window(create_feature_dict_time,seconds_in_bucket = 400, add_suffix = True)
    df_feature_300 = get_stats_window(create_feature_dict_time,seconds_in_bucket = 300, add_suffix = True)
    df_feature_200 = get_stats_window(create_feature_dict_time,seconds_in_bucket = 200, add_suffix = True)
    df_feature_100 = get_stats_window(create_feature_dict_time,seconds_in_bucket = 100, add_suffix = True)

    # Merge all
    df_feature = df_feature.merge(df_feature_500, how = 'left', left_on = 'time_id_', right_on = 'time_id__500')
    df_feature = df_feature.merge(df_feature_400, how = 'left', left_on = 'time_id_', right_on = 'time_id__400')
    df_feature = df_feature.merge(df_feature_300, how = 'left', left_on = 'time_id_', right_on = 'time_id__300')
    df_feature = df_feature.merge(df_feature_200, how = 'left', left_on = 'time_id_', right_on = 'time_id__200')
    df_feature = df_feature.merge(df_feature_100, how = 'left', left_on = 'time_id_', right_on = 'time_id__100')
    # Drop unnecesary time_ids
    df_feature.drop(['time_id__500','time_id__400', 'time_id__300', 'time_id__200','time_id__100'], axis = 1, inplace = True)

    # Create row_id so we can merge later
    stock_id = file_path.split('=')[1]
    df_feature['row_id'] = df_feature['time_id_'].apply(lambda x: f'{stock_id}-{x}')
    df_feature.drop(['time_id_'], axis = 1, inplace = True)

    return df_feature

In [ ]:
# df = book_preprocessor('Data/book_train.parquet/stock_id=0')
# df.describe()

In [ ]:
# df[df['row_id']=='0-5']

#### Trade

In [ ]:
# df = pd.read_parquet('Data/trade_train.parquet/stock_id=1')
# df_ob = pd.read_parquet('Data/book_train.parquet/stock_id=1')
# df = df.groupby('time_id').apply(order_flow_tick)
# df = df.groupby('time_id').apply(order_flow_quote,df_ob=df_ob)
# df = df.droplevel(0).reset_index()
# df = df[df['time_id']==5]
# df_ob = df_ob[df_ob['time_id']==5]

# df_ob['mid_price'] = (df_ob['ask_price1'] + df_ob['bid_price1']) / 2
# overlap = df_ob['seconds_in_bucket'].isin(df['seconds_in_bucket'])
# df['mid_price'] = df_ob.loc[overlap, ['mid_price']].values
# df['order_flow'] = df['size']
# df.loc[df['price']>df['mid_price'],'order_flow'] = -df['order_flow']
# df.loc[df['price']==df['mid_price'],'order_flow'] = 0

In [ ]:
# df_ob

In [ ]:
# volume_per_bar = 50

In [ ]:
# preprocess trade data (for each stock id)
def trade_preprocessor(file_path, file_path_book):
    df = pd.read_parquet(file_path)
    df_ob = pd.read_parquet(file_path_book)

    # calculate log return
    df['log_return'] = df.groupby('time_id')['price'].apply(log_return)
    
    # calculate trade amount
    df['amount']=df['price']*df['size']

    # calculate average volume of each order
    df['average_volume_per_trade'] = df['size'] / df['order_count']

    # calculate trading order flow
    df = df.groupby('time_id').apply(order_flow_tick)
    df = df.groupby('time_id').apply(order_flow_quote,df_ob=df_ob)
    df = df.droplevel(0).reset_index() # drop multi-index created when doing groupby

    # create dict of functions for each feature for aggregations
    create_feature_dict = {
        'log_return':[realized_volatility],
        'seconds_in_bucket':[count_unique],
        'size':[np.sum, np.max, np.min],
        'order_count':[np.sum,np.max],
        'amount':[np.sum,np.max,np.min],
        'average_volume_per_trade':[np.sum,np.max,np.min],
        'order_flow_tick':[np.sum,np.max,np.min],
        'order_flow_quote':[np.sum,np.max,np.min]
    }
    create_feature_dict_time = {
        'log_return':[realized_volatility],
        'seconds_in_bucket':[count_unique],
        'size':[np.sum],
        'order_count':[np.sum],
        'amount':[np.sum,np.max,np.min],
        'average_volume_per_trade':[np.sum,np.max,np.min],
        'order_flow_tick':[np.sum,np.max,np.min],
        'order_flow_quote':[np.sum,np.max,np.min]
    }

    # Function to get group stats for different windows (seconds in bucket)
    def get_stats_window(fe_dict,seconds_in_bucket, add_suffix = False):
        # Group by the window
        df_feature = df[df['seconds_in_bucket'] >= seconds_in_bucket].groupby(['time_id']).agg(fe_dict).reset_index()
        # Rename columns joining suffix
        df_feature.columns = ['_'.join(col) for col in df_feature.columns]
        # Add a suffix to differentiate windows
        if add_suffix:
            df_feature = df_feature.add_suffix('_' + str(seconds_in_bucket))
        return df_feature
    

    # Get the stats for different windows
    df_feature = get_stats_window(create_feature_dict,seconds_in_bucket = 0, add_suffix = False)
    df_feature_500 = get_stats_window(create_feature_dict_time,seconds_in_bucket = 500, add_suffix = True)
    df_feature_400 = get_stats_window(create_feature_dict_time,seconds_in_bucket = 400, add_suffix = True)
    df_feature_300 = get_stats_window(create_feature_dict_time,seconds_in_bucket = 300, add_suffix = True)
    df_feature_200 = get_stats_window(create_feature_dict_time,seconds_in_bucket = 200, add_suffix = True)
    df_feature_100 = get_stats_window(create_feature_dict_time,seconds_in_bucket = 100, add_suffix = True)
    

    # calculate tendency factors
    df_lr = tendency_factors(df) 
    df_feature = df_feature.merge(df_lr, how = 'left', left_on = 'time_id_', right_on = 'time_id')
        
    # Merge all
    df_feature = df_feature.merge(df_feature_500, how = 'left', left_on = 'time_id_', right_on = 'time_id__500')
    df_feature = df_feature.merge(df_feature_400, how = 'left', left_on = 'time_id_', right_on = 'time_id__400')
    df_feature = df_feature.merge(df_feature_300, how = 'left', left_on = 'time_id_', right_on = 'time_id__300')
    df_feature = df_feature.merge(df_feature_200, how = 'left', left_on = 'time_id_', right_on = 'time_id__200')
    df_feature = df_feature.merge(df_feature_100, how = 'left', left_on = 'time_id_', right_on = 'time_id__100')
    # Drop unnecesary time_ids
    df_feature.drop(['time_id__500','time_id__400', 'time_id__300', 'time_id__200','time_id','time_id__100'], axis = 1, inplace = True)
    
    
    df_feature = df_feature.add_prefix('trade_')
    stock_id = file_path.split('=')[1]
    df_feature['row_id'] = df_feature['trade_time_id_'].apply(lambda x:f'{stock_id}-{x}')
    df_feature.drop(['trade_time_id_'], axis = 1, inplace = True)
    return df_feature

In [ ]:
# df = trade_preprocessor('Data/trade_train.parquet/stock_id=0','Data/book_train.parquet/stock_id=0')

In [ ]:
# df[df['row_id']=='0-11']

#### Merge

In [ ]:
# make preprocessing function in parallel (for each stock id)
def preprocessor(list_stock_ids, is_train = True):
    
    # Parrallel for loop
    def for_joblib(stock_id):
        # Train
        if is_train:
            file_path_book = "book_train.parquet/stock_id=" + str(stock_id)
            file_path_trade = "trade_train.parquet/stock_id=" + str(stock_id)
        # Test
        else:
            file_path_book = "book_test.parquet/stock_id=" + str(stock_id)
            file_path_trade = "trade_test.parquet/stock_id=" + str(stock_id)
    
        # Preprocess book and trade data and merge them
        df_tmp = pd.merge(book_preprocessor(file_path_book), trade_preprocessor(file_path_trade,file_path_book), on = 'row_id', how = 'left')
        
        # Return the merge dataframe
        return df_tmp
    
    # Use parallel api to call paralle for loop
    df = Parallel(n_jobs = -1, verbose = 1)(delayed(for_joblib)(stock_id) for stock_id in list_stock_ids)
    # Concatenate all the dataframes that return from Parallel
    df = pd.concat(df, ignore_index = True)
    return df

# get group stats for the stock_id and time_id
def get_time_stock(df):
    vol_cols = ['log_return1_realized_volatility', 'log_return2_realized_volatility', 'log_return1_realized_volatility_400', 'log_return2_realized_volatility_400', 
                'log_return1_realized_volatility_300', 'log_return2_realized_volatility_300', 'log_return1_realized_volatility_200', 'log_return2_realized_volatility_200', 
                'trade_log_return_realized_volatility', 'trade_log_return_realized_volatility_400', 'trade_log_return_realized_volatility_300', 'trade_log_return_realized_volatility_200']


    # Group by the stock id
    df_stock_id = df.groupby(['stock_id'])[vol_cols].agg(['mean', 'std', 'max', 'min', ]).reset_index()
    # Rename columns joining suffix
    df_stock_id.columns = ['_'.join(col) for col in df_stock_id.columns]
    df_stock_id = df_stock_id.add_suffix('_' + 'stock')

    # Group by the stock id
    df_time_id = df.groupby(['time_id'])[vol_cols].agg(['mean', 'std', 'max', 'min', ]).reset_index()
    # Rename columns joining suffix
    df_time_id.columns = ['_'.join(col) for col in df_time_id.columns]
    df_time_id = df_time_id.add_suffix('_' + 'time')
    
    # Merge with original dataframe
    df = df.merge(df_stock_id, how = 'left', left_on = ['stock_id'], right_on = ['stock_id__stock'])
    df = df.merge(df_time_id, how = 'left', left_on = ['time_id'], right_on = ['time_id__time'])
    df.drop(['stock_id__stock', 'time_id__time'], axis = 1, inplace = True)
    return df

### Main

In [ ]:
# Read train and test
train, test = read_train_test()

# Get unique stock ids 
train_stock_ids = train['stock_id'].unique()
# Preprocess them using Parallel and our single stock id functions
train_ = preprocessor(train_stock_ids, is_train = True)
train = train.merge(train_, on = ['row_id'], how = 'left')

# Get unique stock ids 
test_stock_ids = test['stock_id'].unique()
# Preprocess them using Parallel and our single stock id functions
test_ = preprocessor(test_stock_ids, is_train = False)
test = test.merge(test_, on = ['row_id'], how = 'left')

# Get group stats of time_id and stock_id
train = get_time_stock(train)
test = get_time_stock(test)

# train = pd.read_csv(train_file)
# train.drop('Unnamed: 0',axis=1,inplace=True)

In [ ]:
# save the train
train.to_csv(train_file)